In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('iteration4').getOrCreate()

In [2]:
from pyspark.sql.types import (StructField,StringType,IntegerType,FloatType,StructType,DateType,TimestampType)
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import mean
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [3]:
df = spark.read.csv("./Measurement_summary.csv",inferSchema=True,header="true" )

In [4]:
df.show()

+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+
|Measurement_date|Station_code|             Address|  Latitude|  Longitude|  SO2|  NO2|   O3| CO|PM10|PM25|
+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+
|   2017/1/1 0:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.059|0.002|1.2|  73|  57|
|   2017/1/1 1:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.058|0.002|1.2|  71|  59|
|   2017/1/1 2:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|  70|  59|
|   2017/1/1 3:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|  70|  58|
|   2017/1/1 4:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.051|0.002|1.2|  69|  61|
|   2017/1/1 5:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.046|0.002|1.1|  70|  61|
|   2017/1/1 6:00|         1

In [5]:
df.columns

['Measurement_date',
 'Station_code',
 'Address',
 'Latitude',
 'Longitude',
 'SO2',
 'NO2',
 'O3',
 'CO',
 'PM10',
 'PM25']

In [6]:
df.head(2)

[Row(Measurement_date='2017/1/1 0:00', Station_code=101, Address='19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea', Latitude=37.5720164, Longitude=127.0050075, SO2=0.004, NO2=0.059, O3=0.002, CO=1.2, PM10=73, PM25=57),
 Row(Measurement_date='2017/1/1 1:00', Station_code=101, Address='19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea', Latitude=37.5720164, Longitude=127.0050075, SO2=0.004, NO2=0.058, O3=0.002, CO=1.2, PM10=71, PM25=59)]

In [7]:
df.count()

647511

In [8]:
df.printSchema()

root
 |-- Measurement_date: string (nullable = true)
 |-- Station_code: integer (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- SO2: double (nullable = true)
 |-- NO2: double (nullable = true)
 |-- O3: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- PM10: integer (nullable = true)
 |-- PM25: integer (nullable = true)



In [9]:
df.describe().show()

+-------+----------------+------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-----------------+------------------+
|summary|Measurement_date|      Station_code|             Address|           Latitude|          Longitude|                 SO2|                NO2|                  O3|                 CO|             PM10|              PM25|
+-------+----------------+------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-----------------+------------------+
|  count|          647511|            647511|              647511|             647511|             647511|              647294|             647511|              647511|             647511|           647511|            647511|
|   mean|            null|113.00022084566903|                null|  37.55348357387394| 126.98933

In [12]:
plot1=df.groupBy("PM25").count().orderBy("PM25")

In [14]:
plot1.count()

333

In [15]:
plot1.show(333)

+----+-----+
|PM25|count|
+----+-----+
|  -1| 3973|
|   0| 6123|
|   1| 7798|
|   2| 5266|
|   3|13007|
|   4|11094|
|   5|12765|
|   6|15299|
|   7|15733|
|   8|17073|
|   9|17915|
|  10|19349|
|  11|19669|
|  12|20827|
|  13|20367|
|  14|20844|
|  15|20619|
|  16|21238|
|  17|21294|
|  18|19762|
|  19|18914|
|  20|18049|
|  21|17262|
|  22|16677|
|  23|15306|
|  24|14692|
|  25|13873|
|  26|13186|
|  27|12831|
|  28|11525|
|  29|10722|
|  30|10335|
|  31| 9423|
|  32| 8984|
|  33| 8337|
|  34| 8010|
|  35| 7211|
|  36| 6926|
|  37| 6501|
|  38| 6518|
|  39| 5608|
|  40| 5445|
|  41| 4902|
|  42| 4659|
|  43| 4260|
|  44| 4178|
|  45| 3766|
|  46| 3586|
|  47| 3358|
|  48| 3288|
|  49| 2955|
|  50| 2760|
|  51| 2548|
|  52| 2480|
|  53| 2302|
|  54| 2212|
|  55| 2034|
|  56| 1983|
|  57| 1812|
|  58| 1713|
|  59| 1789|
|  60| 1601|
|  61| 1511|
|  62| 1479|
|  63| 1285|
|  64| 1310|
|  65| 1130|
|  66| 1150|
|  67| 1061|
|  68| 1075|
|  69|  991|
|  70|  896|
|  71|  888|
|  72|  837|

In [32]:
df.select([count(when(isnan(c)|col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------------+------------+-------+--------+---------+---+---+---+---+----+----+
|Measurement_date|Station_code|Address|Latitude|Longitude|SO2|NO2| O3| CO|PM10|PM25|
+----------------+------------+-------+--------+---------+---+---+---+---+----+----+
|               0|           0|      0|       0|        0|217|  0|  0|  0|   0|   0|
+----------------+------------+-------+--------+---------+---+---+---+---+----+----+



In [22]:
quantiles = df.approxQuantile("PM25", [0.25,0.998], 0.00)

In [23]:
quantiles[1]

171.0

In [25]:
quantiles10 = df.approxQuantile("PM25", [0.25,0.99], 0.00)
quantiles10[1]

100.0

In [27]:
df2=df.filter("Station_code=101")

In [33]:
df2.count()

25905

In [34]:
df2.show(5)

+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+
|Measurement_date|Station_code|             Address|  Latitude|  Longitude|  SO2|  NO2|   O3| CO|PM10|PM25|
+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+
|   2017/1/1 0:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.059|0.002|1.2|  73|  57|
|   2017/1/1 1:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.058|0.002|1.2|  71|  59|
|   2017/1/1 2:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|  70|  59|
|   2017/1/1 3:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|  70|  58|
|   2017/1/1 4:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.051|0.002|1.2|  69|  61|
+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+
only showing top 5 rows



In [35]:
df2.select([count(when(isnan(c)|col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+----------------+------------+-------+--------+---------+---+---+---+---+----+----+
|Measurement_date|Station_code|Address|Latitude|Longitude|SO2|NO2| O3| CO|PM10|PM25|
+----------------+------------+-------+--------+---------+---+---+---+---+----+----+
|               0|           0|      0|       0|        0| 26|  0|  0|  0|   0|   0|
+----------------+------------+-------+--------+---------+---+---+---+---+----+----+



In [36]:
mean_SO2 = df2.select(mean(df2['SO2'])).collect()

In [37]:
mean_SO2[0][0]

0.004308126279995904

In [38]:
mean_SO2_val = mean_SO2[0][0]

In [39]:
df3=df2.na.fill(mean_SO2_val, subset=['SO2'])

In [40]:
df3.select([count(when(isnan(c)|col(c).isNull(), c)).alias(c) for c in df3.columns]).show()

+----------------+------------+-------+--------+---------+---+---+---+---+----+----+
|Measurement_date|Station_code|Address|Latitude|Longitude|SO2|NO2| O3| CO|PM10|PM25|
+----------------+------------+-------+--------+---------+---+---+---+---+----+----+
|               0|           0|      0|       0|        0|  0|  0|  0|  0|   0|   0|
+----------------+------------+-------+--------+---------+---+---+---+---+----+----+



In [41]:
df3.describe().show()

+-------+----------------+------------+--------------------+-----------------+------------------+--------------------+--------------------+--------------------+------------------+-----------------+------------------+
|summary|Measurement_date|Station_code|             Address|         Latitude|         Longitude|                 SO2|                 NO2|                  O3|                CO|             PM10|              PM25|
+-------+----------------+------------+--------------------+-----------------+------------------+--------------------+--------------------+--------------------+------------------+-----------------+------------------+
|  count|           25905|       25905|               25905|            25905|             25905|               25905|               25905|               25905|             25905|            25905|             25905|
|   mean|            null|       101.0|                null|37.57201640002134|127.00500749994949|0.004308126279995...|0.030516348195

In [43]:
mean_CO=df3.select(mean(df2['CO'])).collect()

In [44]:
mean_PM10=df3.select(mean(df2['PM10'])).collect()

In [45]:
mean_PM25=df3.select(mean(df2['PM25'])).collect()

In [46]:
mean_NO2=df3.select(mean(df2['NO2'])).collect()

In [47]:
mean_O3=df3.select(mean(df2['O3'])).collect()

In [48]:
df4 = df3.withColumn("SO2", \
              when(df3["SO2"] == -1,mean_SO2_val ).otherwise(df3["SO2"])).withColumn("NO2", \
              when(df3["NO2"] == -1,mean_NO2[0][0] ).otherwise(df3["NO2"])).withColumn("O3", \
              when(df3["O3"] == -1,mean_O3[0][0] ).otherwise(df3["O3"])).withColumn("CO", \
              when(df3["CO"] == -1,mean_CO[0][0] ).otherwise(df3["CO"])).withColumn("PM10", \
              when(df3["PM10"] == -1,mean_PM10[0][0] ).otherwise(df3["PM10"])).withColumn("PM25", \
              when(df3["PM25"] == -1,mean_PM25[0][0] ).otherwise(df3["PM25"]))

In [49]:
df4.describe("SO2","NO2","PM10","PM25","CO","O3").show(truncate = False)

+-------+--------------------+--------------------+------------------+------------------+------------------+--------------------+
|summary|SO2                 |NO2                 |PM10              |PM25              |CO                |O3                  |
+-------+--------------------+--------------------+------------------+------------------+------------------+--------------------+
|count  |25905               |25905               |25905             |25905             |25905             |25905               |
|mean   |0.004385664062376209|0.03163020489285626 |38.01073028963004 |22.889122526553802|0.5854755987284198|0.024624145914049864|
|stddev |0.004811476912778703|0.016596226285800948|27.395879348007718|19.982107179194774|0.4781151915162288|0.01910201575997793 |
|min    |0.0                 |0.0                 |0.0               |0.0               |0.0               |0.0                 |
|max    |0.406               |0.109               |516.0             |513.0             |4

In [78]:
quantiles = df4.approxQuantile("PM25", [0.25,0.987], 0.0)
print(quantiles[1],quantiles[0])

85.0 11.0


In [79]:
IQR = quantiles[1] - quantiles[0]
bounds = {}
bounds["PM25"] = [quantiles[0] - 1.5*IQR, quantiles[1] + 1.5*IQR]
print("bounds: ",bounds)

bounds:  {'PM25': [-100.0, 196.0]}


In [81]:
quantiles2 = df4.approxQuantile("PM10", [0.25,0.96], 0.0)
print(quantiles2[1],quantiles2[0])

92.0 21.0


In [82]:
IQR2 = quantiles2[1] - quantiles2[0]
bounds = {}
bounds["PM10"] = [quantiles2[0] - 1.5*IQR2, quantiles2[1] + 1.5*IQR2]
print("bounds: ",bounds)

bounds:  {'PM10': [-85.5, 198.5]}


In [83]:
df5 = df4.withColumn("PM10", \
              when(df4["PM10"] >199,mean_PM10[0][0] ).otherwise(df4["PM10"])).withColumn("PM25", \
              when(df4["PM25"] >196,mean_PM25[0][0] ).otherwise(df4["PM25"]))

In [85]:
df5.describe("PM10","PM25").show()

+-------+------------------+------------------+
|summary|              PM10|              PM25|
+-------+------------------+------------------+
|  count|             25905|             25905|
|   mean| 37.51481102558707|22.561284814181356|
| stddev|24.751142376615665| 17.37208797651781|
|    min|               0.0|               0.0|
|    max|             199.0|             191.0|
+-------+------------------+------------------+



In [86]:
from pyspark.sql.functions import monotonically_increasing_id

In [87]:
df6 = df5.withColumn("id", monotonically_increasing_id())

In [88]:
df6.show()

+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+
|Measurement_date|Station_code|             Address|  Latitude|  Longitude|  SO2|  NO2|   O3| CO|PM10|PM25| id|
+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+
|   2017/1/1 0:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.059|0.002|1.2|73.0|57.0|  0|
|   2017/1/1 1:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.058|0.002|1.2|71.0|59.0|  1|
|   2017/1/1 2:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|59.0|  2|
|   2017/1/1 3:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|58.0|  3|
|   2017/1/1 4:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.051|0.002|1.2|69.0|61.0|  4|
|   2017/1/1 5:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.046|0.002|1.1|70.0|61

In [89]:
my_window = Window.partitionBy().orderBy("id")
df6=df6.withColumn("pre",F.lag(df6.PM25).over(my_window))

In [90]:
df6.show()

+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+----+
|Measurement_date|Station_code|             Address|  Latitude|  Longitude|  SO2|  NO2|   O3| CO|PM10|PM25| id| pre|
+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+----+
|   2017/1/1 0:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.059|0.002|1.2|73.0|57.0|  0|null|
|   2017/1/1 1:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.058|0.002|1.2|71.0|59.0|  1|57.0|
|   2017/1/1 2:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|59.0|  2|59.0|
|   2017/1/1 3:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|58.0|  3|59.0|
|   2017/1/1 4:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.051|0.002|1.2|69.0|61.0|  4|58.0|
|   2017/1/1 5:00|         101|19, Jong-ro 35ga-...|37.5720164|1

In [91]:
df6=df6.withColumn("diff",F.when(F.isnull(df6.PM25-df6.pre),0).otherwise(df6.PM25-df6.pre))

In [92]:
df6.show()

+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+----+----+
|Measurement_date|Station_code|             Address|  Latitude|  Longitude|  SO2|  NO2|   O3| CO|PM10|PM25| id| pre|diff|
+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+----+----+
|   2017/1/1 0:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.059|0.002|1.2|73.0|57.0|  0|null| 0.0|
|   2017/1/1 1:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.058|0.002|1.2|71.0|59.0|  1|57.0| 2.0|
|   2017/1/1 2:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|59.0|  2|59.0| 0.0|
|   2017/1/1 3:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|58.0|  3|59.0|-1.0|
|   2017/1/1 4:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.051|0.002|1.2|69.0|61.0|  4|58.0| 3.0|
|   2017/1/1 5:00|      

In [93]:
df_s = spark.read.csv("./status5.csv",header="true" )

In [97]:
df_s.show(2)
df_s.printSchema()
df_s.count()

+----------------+------+
|Measurement_date|status|
+----------------+------+
|   2017/1/1 0:00|     0|
|   2017/1/1 1:00|     0|
+----------------+------+
only showing top 2 rows

root
 |-- Measurement_date: string (nullable = true)
 |-- status: string (nullable = true)



25905

In [98]:
df_n = df6.join(df_s, df5.Measurement_date == df_s.Measurement_date,how='left').drop(df_s.Measurement_date)

In [100]:
df_n.show(5)

+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+----+----+------+
|Measurement_date|Station_code|             Address|  Latitude|  Longitude|  SO2|  NO2|   O3| CO|PM10|PM25| id| pre|diff|status|
+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+----+----+------+
|   2017/1/1 0:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.059|0.002|1.2|73.0|57.0|  0|null| 0.0|     0|
|   2017/1/1 1:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.058|0.002|1.2|71.0|59.0|  1|57.0| 2.0|     0|
|   2017/1/1 2:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|59.0|  2|59.0| 0.0|     0|
|   2017/1/1 3:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|58.0|  3|59.0|-1.0|     0|
|   2017/1/1 4:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.051|0.002|1.2|

In [101]:
df_n.count()

25905

In [102]:
df_n.printSchema()

root
 |-- Measurement_date: string (nullable = true)
 |-- Station_code: integer (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- SO2: double (nullable = false)
 |-- NO2: double (nullable = true)
 |-- O3: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- PM10: double (nullable = true)
 |-- PM25: double (nullable = true)
 |-- id: long (nullable = false)
 |-- pre: double (nullable = true)
 |-- diff: double (nullable = true)
 |-- status: string (nullable = true)



In [103]:
from pyspark.sql.functions import unix_timestamp, col
from pyspark.sql.types import TimestampType

In [105]:
df7 = df_n.select(unix_timestamp(df_n.Measurement_date, 'yyyy/MM/dd HH:mm').cast(TimestampType()).alias("timestamp"),
                  "Station_code","Address","Latitude","Longitude","SO2","NO2","O3","CO","PM10","PM25","id","diff","status")
df7.show()

+--------------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+----+------+
|           timestamp|Station_code|             Address|  Latitude|  Longitude|  SO2|  NO2|   O3| CO|PM10|PM25| id|diff|status|
+--------------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+----+---+----+------+
|2017-01-01 00:00:...|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.059|0.002|1.2|73.0|57.0|  0| 0.0|     0|
|2017-01-01 01:00:...|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.058|0.002|1.2|71.0|59.0|  1| 2.0|     0|
|2017-01-01 02:00:...|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|59.0|  2| 0.0|     0|
|2017-01-01 03:00:...|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0|58.0|  3|-1.0|     0|
|2017-01-01 04:00:...|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.051|0.002|1.2|69.

In [106]:
df7.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- Station_code: integer (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- SO2: double (nullable = false)
 |-- NO2: double (nullable = true)
 |-- O3: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- PM10: double (nullable = true)
 |-- PM25: double (nullable = true)
 |-- id: long (nullable = false)
 |-- diff: double (nullable = true)
 |-- status: string (nullable = true)



In [120]:
selectdata=df7.select("SO2","NO2","O3","CO","PM10","PM25","diff","status")

In [122]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [123]:
stringIndexer = StringIndexer(inputCol="status", outputCol="statusIndex")
model = stringIndexer.fit(selectdata)
indexed = model.transform(selectdata)

In [124]:
encoder = OneHotEncoder(inputCol="statusIndex", outputCol="statusVec")
encoded = encoder.transform(indexed)

In [121]:
plot2=selectdata.groupBy("status").count().orderBy("status")
plot2.show()

+------+-----+
|status|count|
+------+-----+
|     0|25362|
|     1|  172|
|     2|  118|
|     4|   43|
|     8|   64|
|     9|  146|
+------+-----+



In [125]:
encoded.show()

+-----+-----+-----+---+----+----+----+------+-----------+-------------+
|  SO2|  NO2|   O3| CO|PM10|PM25|diff|status|statusIndex|    statusVec|
+-----+-----+-----+---+----+----+----+------+-----------+-------------+
|0.004|0.059|0.002|1.2|73.0|57.0| 0.0|     0|        0.0|(5,[0],[1.0])|
|0.004|0.058|0.002|1.2|71.0|59.0| 2.0|     0|        0.0|(5,[0],[1.0])|
|0.004|0.056|0.002|1.2|70.0|59.0| 0.0|     0|        0.0|(5,[0],[1.0])|
|0.004|0.056|0.002|1.2|70.0|58.0|-1.0|     0|        0.0|(5,[0],[1.0])|
|0.003|0.051|0.002|1.2|69.0|61.0| 3.0|     0|        0.0|(5,[0],[1.0])|
|0.003|0.046|0.002|1.1|70.0|61.0| 0.0|     0|        0.0|(5,[0],[1.0])|
|0.003|0.049|0.002|1.1|66.0|57.0|-4.0|     0|        0.0|(5,[0],[1.0])|
|0.003|0.045|0.002|1.0|71.0|60.0| 3.0|     0|        0.0|(5,[0],[1.0])|
|0.004|0.047|0.002|1.1|72.0|60.0| 0.0|     0|        0.0|(5,[0],[1.0])|
|0.003|0.047|0.002|1.1|74.0|63.0| 3.0|     0|        0.0|(5,[0],[1.0])|
|0.004|0.045|0.004|1.0|76.0|68.0| 5.0|     0|        0.0|(5,[0],

In [127]:
assembler = VectorAssembler(
    inputCols=["SO2", "NO2","O3","CO","PM10","diff","statusVec"],
    outputCol="features")

In [128]:
output = assembler.transform(encoded)

In [129]:
output.printSchema()
output.head(1)

root
 |-- SO2: double (nullable = false)
 |-- NO2: double (nullable = true)
 |-- O3: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- PM10: double (nullable = true)
 |-- PM25: double (nullable = true)
 |-- diff: double (nullable = true)
 |-- status: string (nullable = true)
 |-- statusIndex: double (nullable = true)
 |-- statusVec: vector (nullable = true)
 |-- features: vector (nullable = true)



[Row(SO2=0.004, NO2=0.059, O3=0.002, CO=1.2, PM10=73.0, PM25=57.0, diff=0.0, status='0', statusIndex=0.0, statusVec=SparseVector(5, {0: 1.0}), features=SparseVector(11, {0: 0.004, 1: 0.059, 2: 0.002, 3: 1.2, 4: 73.0, 6: 1.0}))]

In [130]:
final_data = output.select("features",'PM25')
final_data.show()

+--------------------+----+
|            features|PM25|
+--------------------+----+
|(11,[0,1,2,3,4,6]...|57.0|
|[0.004,0.058,0.00...|59.0|
|(11,[0,1,2,3,4,6]...|59.0|
|[0.004,0.056,0.00...|58.0|
|[0.003,0.051,0.00...|61.0|
|(11,[0,1,2,3,4,6]...|61.0|
|[0.003,0.049,0.00...|57.0|
|[0.003,0.045,0.00...|60.0|
|(11,[0,1,2,3,4,6]...|60.0|
|[0.003,0.047,0.00...|63.0|
|[0.004,0.045,0.00...|68.0|
|[0.004,0.048,0.00...|74.0|
|[0.004,0.054,0.00...|79.0|
|(11,[0,1,2,3,4,6]...|79.0|
|[0.006,0.055,0.00...|78.0|
|[0.006,0.056,0.00...|69.0|
|(11,[0,1,2,3,4,6]...|69.0|
|[0.005,0.063,0.00...|72.0|
|[0.005,0.063,0.00...|69.0|
|[0.005,0.062,0.00...|70.0|
+--------------------+----+
only showing top 20 rows



In [131]:
train_data,test_data = final_data.randomSplit([0.8,0.2])

In [132]:
train_data.show()
train_data.describe().show()

+--------------------+------------------+
|            features|              PM25|
+--------------------+------------------+
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|22.850453580389885|
|(11,[0,1,2,3,4],[...|              12.0|
|(11,[0,1,2,3,4],[...|              20.0|
|(11,[0,1,2,3,4],[...|              20.0|
+--------------------+------------

In [133]:
test_data.show()
test_data.describe().show()

+--------------------+----+
|            features|PM25|
+--------------------+----+
|(11,[0,1,2,3,4],[...|14.0|
|(11,[0,1,2,3,4,5]...|17.0|
|(11,[0,1,2,3,4,5]...|17.0|
|(11,[0,1,2,3,4,5]...|19.0|
|(11,[0,1,2,3,4,5]...|21.0|
|(11,[0,1,2,3,4,6]...|13.0|
|(11,[0,1,2,3,4,6]...|14.0|
|(11,[0,1,2,3,4,6]...|15.0|
|(11,[0,1,2,3,4,6]...| 6.0|
|(11,[0,1,2,3,4,6]...|13.0|
|(11,[0,1,2,3,4,6]...|27.0|
|(11,[0,1,2,3,4,6]...| 8.0|
|(11,[0,1,2,3,4,6]...| 7.0|
|(11,[0,1,2,3,4,6]...|11.0|
|(11,[0,1,2,3,4,6]...| 3.0|
|(11,[0,1,2,3,4,6]...| 3.0|
|(11,[0,1,2,3,4,6]...| 3.0|
|(11,[0,1,2,3,4,6]...| 3.0|
|(11,[0,1,2,3,4,6]...| 3.0|
|(11,[0,1,2,3,4,6]...| 2.0|
+--------------------+----+
only showing top 20 rows

+-------+------------------+
|summary|              PM25|
+-------+------------------+
|  count|              5207|
|   mean| 22.14048483499211|
| stddev|16.564878834415612|
|    min|               0.0|
|    max|             152.0|
+-------+------------------+



In [135]:
from pyspark.ml.regression import LinearRegression

In [136]:
lr = LinearRegression(featuresCol='features', labelCol='PM25', predictionCol='prediction')

In [137]:
correct_model = lr.fit(train_data)

In [138]:
test_results = correct_model.evaluate(test_data)

In [139]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))
# Print R-Squared.
print("r2: {}".format(test_results.r2))

+--------------------+
|           residuals|
+--------------------+
|-0.44294349331080696|
| -2.2124195078696225|
|  3.6225289006525117|
|  0.6986145328581017|
|   3.827793981178754|
|   2.223991798767269|
|   3.046070923180743|
|   2.709150618192133|
|  0.9351996421338598|
|  0.9583694873424804|
| -1.7946169007890518|
|  -8.466617304159925|
|  2.6441795834660162|
|   3.120826163053348|
|   0.363821787977741|
|  0.7867807600754944|
|  0.2580910110079606|
|  2.3289481415271593|
|   1.800258392459626|
| -3.3993137879675537|
+--------------------+
only showing top 20 rows

RMSE: 8.17831189113407
r2: 0.7561997061521781


In [140]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(correct_model.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(correct_model.intercept)))

Coefficients: [-583.9839613150671,117.81947055130345,43.90186575093384,7.640224915163791,0.528689749067533,0.32705007941266045,-2.215660469893606,4.8232533607720445,-9.156436241266865,-1.7945020711628836,-4.054594425647106]


Intercept:-1.7617990937992114


In [108]:
df7.stat.corr("SO2","PM25")

0.17282307850456896

In [111]:
df7.stat.corr("NO2","PM25")

0.5274193953698306

In [112]:
df7.stat.corr("CO","PM25")

0.35807770994631544

In [113]:
df7.stat.corr("O3","PM25")

-0.03001099285915479

In [115]:
df7.stat.corr("Latitude","PM25")

nan

In [116]:
df7.stat.corr("Station_code","PM25")

nan

In [117]:
df7.stat.corr("Longitude","PM25")

nan

In [118]:
df7.stat.corr("PM10","PM25")

0.8500864132405491

In [119]:
df7.stat.corr("diff","PM25")

0.18482124840808153

In [126]:
encoded.printSchema()

root
 |-- SO2: double (nullable = false)
 |-- NO2: double (nullable = true)
 |-- O3: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- PM10: double (nullable = true)
 |-- PM25: double (nullable = true)
 |-- diff: double (nullable = true)
 |-- status: string (nullable = true)
 |-- statusIndex: double (nullable = true)
 |-- statusVec: vector (nullable = true)

